In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import requests
import numpy as np
import pandas as pd
import time
import threading


#the goal of datasaver is to gather all data in one place and send it to the google sheet after the test
# 1. Second window creates a dictionary where username, date of birth and email are saved
# 2. First: username, date of birth and email is saved. not all of these are needed to proceed and NaN there is okay.
# 3. 2D numpy array is made for storing the choice, index of choice, correctness of choice and the time taken for the choice
# 4. Each time reply is recorded, the choice, index of the choice, correctness of the choice and the time taken for the choice is saved in a 2D numpy array
# 3. 2D numpy array is transposed and three more columns are made, populated with name, username, date of birth and email up to the index of the choice
# 4. This is then saved into google sheets using an API

class DataSaver:
    def __init__(self):
        self.credentials = {}
        self.letter_answers_df = pd.DataFrame(columns=["button_clicked", "correctness", "time_taken"])

    # def initialize_google_sheets(self):
    #     SERVICE_ACCOUNT_FILE = 'json file path'
    #     SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    #     credentials = Credentials.from_service_account_file(
    #         SERVICE_ACCOUNT_FILE, scopes=SCOPES
    #     )
    #     return build('sheets', 'v4', credentials=credentials)

    # def send_data_to_sheets(self):
    #     # Initialize Google Sheets API
    #     service = self.initialize_google_sheets()
    #     sheet = service.spreadsheets()
    
    #     try:
    #         # Replace NaN values with "not_submitted"
    #         clean_df = self.letter_answers_df.fillna("not_submitted")
    #         data_to_send = clean_df.values.tolist()
    
    #         # Get the existing data to check the structure
    #         existing_data = sheet.values().get(spreadsheetId=self.SPREADSHEET_ID, range="Sheet1").execute()
    #         existing_values = existing_data.get('values', [])
            
    #         # Check if the sheet is empty or if columns need to be created
    #         if not existing_values or existing_values[0] != ["button_clicked", "correctness", "time_taken", "name_surname", "age", "email"]:
    #             # Create or reset the header
    #             sheet.values().update(
    #                 spreadsheetId=self.SPREADSHEET_ID, 
    #                 range="Sheet1", 
    #                 body={"values": [["button_clicked", "correctness", "time_taken", "name_surname", "age", "email"]]},
    #                 valueInputOption="RAW"
    #             ).execute()
    
    #         # Append new data
    #         body = {
    #             "values": data_to_send
    #         }
    #         sheet.values().append(
    #             spreadsheetId=self.SPREADSHEET_ID,
    #             range="Sheet1",
    #             body=body,
    #             valueInputOption="RAW"
    #         ).execute()
    
    #     except HttpError as err:
    #         print(f"An error occurred: {err}")

    def letter_answers(self, button_clicked, correctness, time_taken):
        # Use the credentials directly in the new row data, using "not_submitted" if the data is not provided
        new_row_data = {
            "button_clicked": button_clicked,
            "correctness": correctness,
            "time_taken": time_taken
        }
    
        # Create a new DataFrame from new_row_data and append it
        new_row = pd.DataFrame([new_row_data])
        self.letter_answers_df = pd.concat([self.letter_answers_df, new_row], ignore_index=True)

    def send_to_forms(self):
        # Convert the DataFrame to a JSON formatted string
        all_answers_json = self.letter_answers_df.to_json(orient='records')

        # Prepare data dictionary
        data_dict = {
            'participant_name': self.credentials.get("name_surname", "not_submitted"),
            'age': self.credentials.get("age", "not_submitted"),
            'email': self.credentials.get("email", "not_submitted"),
            'test_results': all_answers_json
        }

        # URL of your Google Form submission script
        form_url = "https://docs.google.com/forms/d/e/1FAIpQLSd02cVHITeCE3IXkhJf3SvSlCmGbLqnQP1duvIPtpQfU7k14A/formResponse"

        # Prepare the data for the Google Form
        form_data = {
        'entry.939488502': data_dict['participant_name'],
        'entry.1067383362': data_dict['age'],
        'entry.1376612259': data_dict['email'],
        'entry.1009968932': data_dict['test_results']  #the JSON string with data
    }

        # Send the data to the Google Form
        response = requests.post(form_url, data=form_data)
        return response.ok

#timer class has two major functionalities:
    # - it counts how many seconds user spends on a puzzle, which will be saved as data
    # - it counts how many seconds user spent on the puzzles overall, to prevent further guessing after a timeout
class Timer:
    def __init__(self, window_manager):
        self.puzzle_timer_count = 0
        self.test_timer_count = 300
        self.window_manager = window_manager
        self.test_timer_widget = widgets.Label()
        self.puzzle_timer_thread = None

    def puzzle_timer_manager(self, startstop):
        if startstop == "start":
            self.puzzle_timer_count = 0  # Reset timer for the new puzzle
            if self.puzzle_timer_thread is None or not self.puzzle_timer_thread.is_alive():
                self.puzzle_timer_thread = threading.Thread(target=self.puzzle_timer)
                self.puzzle_timer_thread.start()
        elif startstop == "stop":
            if self.puzzle_timer_thread is not None:
                # Capture the time taken before stopping the thread
                self.time_taken = self.puzzle_timer_count
                # Stop the timer thread
                self.puzzle_timer_thread = None
                self.puzzle_timer_count = 0

    def puzzle_timer(self):
        while self.puzzle_timer_thread is not None:
            time.sleep(1)
            self.puzzle_timer_count += 1


    def start_test_timer(self):
        self.test_timer_thread = threading.Thread(target=self.test_timer)
        self.test_timer_thread.start()
        
    def test_timer(self):
        while self.test_timer_count > 0:
            time.sleep(1)
            self.test_timer_count -= 1
            self.update_test_timer_widget()
        self.window_manager.end_test()

    def update_test_timer_widget(self):
        if self.test_timer_count > 0:
            new_label = f"{self.test_timer_count} seconds left"
        else: 
            self.window_manager.current_window = 8
            new_label = "Time's up!"
        self.test_timer_widget.value = new_label
            
        
#creates a class for a window manager which manages which window is shown at any given time.
#it begins with window 1 and stores the information about what window is being used at a given time.
#it also contains logic for going backwards and forwards in windows
class WindowManager:
    def __init__(self):
        self.current_window = 1
        self.output_panel = widgets.Output(layout={"border": "1px solid black", "height":"600px", "width":"1000px"})
        display(self.output_panel)
        self.panels = Panels(self)

    def panelchanger(self, button_input):
        if button_input == 1:
            self.current_window += 1
        elif button_input == 0:
            self.current_window -= 1
        print(f"current wondow: {self.current_window}")
        self.whichpanel()

    def whichpanel(self):
        with self.output_panel:
            clear_output()
            if self.current_window == 1:
                self.panels.start_panel()
            if self.current_window == 2:
                self.panels.credentials_panel()
            elif self.current_window >= 3 and self.current_window <= 7:
                self.panels.puzzle_panel()
            elif self.current_window > 7:
                self.panels.stop_panel()


#creates a class for different panels to be displayed
class Panels:
    def __init__(self, window_manager):
        self.window_manager = window_manager
        self.puzzle_creator = PuzzleCreator()
        self.timer = Timer(window_manager)
        self.data_saver = DataSaver()

    #This creates the contents of the start panel to be displayed on output_panel canvas
    def start_panel(self):
        def startbuttonfunction(btn):
            self.window_manager.panelchanger(1)
            self.timer.start_test_timer()

        with self.window_manager.output_panel:
            clear_output(wait=True)

            # Define welcome text
            # Define welcome text with centered alignment
            welcome_text = widgets.HTML(
                value="<div style='text-align: center;'>"
                      "<h2>Thank you for wanting to participate in our spatial reasoning task!</h2>"
                      "<p>In this test, you will have to determine, which two dimensional view of a figure does NOT correspond<br>"
                      "to a 3D view of a figure. You will have 5 minutes to answer all five questions. You will not be allowed<br>"
                      "to stop the timer or alter your choices, once they were submitted. Your results are going to<br>"
                      "show up at the end of the quiz. Example on how to answer a question below. <b>Good luck!</b></p>"
                      "</div>",
                layout=widgets.Layout(justify_content='center')
            )

            # Load and display the tutorial image
            image_path = "tutorial_photo.png"
            file = open(image_path, "rb")
            image = file.read()
            tutorial_photo = widgets.Image(
                value=image,
                format='png',
                width='563',
                height='338',  # Adjust the height according to your preference
            )

            # Define the start button
            start_button = widgets.Button(
                description="Start the test",
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                layout=widgets.Layout(width='auto', height='auto'),
                style={'button_color': '#042940', 'font_weight': 'bold'}
            )
            start_button.on_click(startbuttonfunction)

            # Organize widgets in a VBox
            vbox = widgets.VBox([welcome_text, tutorial_photo, start_button],
                                layout=widgets.Layout(align_items='center', justify_content='space-around'))

            # Display everything
            display(vbox)

    def credentials_panel(self):

        def submit_functionality(btn):
            name_surname = name_surname_text.value
            age = age_text.value
            email = email_text.value
            self.data_saver.credentials = {"name_surname": name_surname,"age": age,"email": email}
            self.window_manager.panelchanger(1)
            print("Changing to new panel")
    
        with self.window_manager.output_panel:
            clear_output()
    
            # Instruction text
            instructions_html = widgets.HTML(
                value="<div style='text-align: center;'>"
                      "<p><b>Before you begin, please consider submitting your credentials.</b><br>"
                      "Your age will be used for deepening our analysis of the results, and your name and surname<br>"
                      "and email will be only used to debrief you about our findings in the future. You do not have to<br>"
                      "provide any data if you don't wish to, in which case your answers will remain fully anonymous.</p>"
                      "</div>",
                layout=widgets.Layout(justify_content='center')
            )
    
            # Input fields with bold labels
            name_surname_text = widgets.Text(placeholder="Enter your name and surname")
            age_text = widgets.Text(placeholder="Enter your age")
            email_text = widgets.Text(placeholder="Enter your email")
    
            # Submit button
            submit_button = widgets.Button(
                description="Submit",
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                layout=widgets.Layout(width='auto', height='auto'),
                style={'button_color': '#042940', 'font_weight': 'bold'}
            )
            submit_button.on_click(submit_functionality)
    
            # Organize widgets in a VBox
            vbox = widgets.VBox([instructions_html,
                                 widgets.HTML(value="<b>Name and surname:</b>"),
                                 name_surname_text,
                                 widgets.HTML(value="<b>Age:</b>"),
                                 age_text,
                                 widgets.HTML(value="<b>Email:</b>"),
                                 email_text,
                                 submit_button],
                                layout=widgets.Layout(align_items='center', justify_content='space-around'))
    
            # Display everything
            display(vbox)

            
    def stop_panel(self):
        # Calculate the number of rows to consider for the sum
        num_rows_to_consider = len(self.data_saver.letter_answers_df)
        print("Updated DataFrame:")
        print(self.data_saver.letter_answers_df)
        # Assuming 'correctness' data is in self.letter_answers_df
        # and it's a column with numeric values
        # Get the sum of the last 'num_rows_to_consider' rows in 'correctness' column
        
        sum_of_correctness = self.data_saver.letter_answers_df['correctness'].tail(num_rows_to_consider).sum()
        print("sum_of_correctness")
        sum_of_correctness = self.data_saver.letter_answers_df['correctness'].sum()
        print(f"Sum of correctness: {sum_of_correctness}")
        
        with self.window_manager.output_panel:
            clear_output()
    
            # Display the thank you message and score
            farewell_html = widgets.HTML(
                value=f"<div style='text-align: center; font-size: 20px;'>"
                      f"<p><b>Thank you for participation!</b></p>"
                      f"<p>Your Score was <span style='color: #005C53;'>{sum_of_correctness}</span> out of <span style='color: #005C53;'>5</span>.</p>"
                      f"</div>",
                layout=widgets.Layout(justify_content='center')
            )
            display(farewell_html)
    
       #send to google forms
        self.data_saver.send_to_forms()


    #This creates the contents of the puzzle panels to be displayed on the output_panel canvas
    def puzzle_panel(self):

        def handle_button_click(letter):
            def button_click_handler(btn):
                self.timer.puzzle_timer_manager("stop") # Stop the timer and get the time taken
                time_taken = self.timer.time_taken 
                self.data_saver.letter_answers(letter, self.puzzle_creator.correctness[letter], time_taken) # Record the answer and time taken
                self.window_manager.panelchanger(1) # Change to the next panel
         
            return button_click_handler

        with self.window_manager.output_panel:
            clear_output()

            #create the figure and axes to be populated with plots
            fig = plt.figure(figsize=(12,6))
            gs = gridspec.GridSpec(2, 4, figure=fig)
            axes = [
                fig.add_subplot(gs[0:2, 0:2], projection='3d'),  # Main 3D image to be guessed
                fig.add_subplot(gs[0, 2], projection='3d'),      # projection A
                fig.add_subplot(gs[0, 3], projection='3d'),      # projection B
                fig.add_subplot(gs[1, 2], projection='3d'),      # projection C
                fig.add_subplot(gs[1, 3], projection='3d')       # projection D
                ]

            #adjusts the puzzle number to be in sequence
            puzzle_number = self.window_manager.current_window - 2

            # Call create_puzzle method from puzzle creator class, populating the figure
            self.puzzle_creator.create_puzzle(puzzle_number, axes)
            plt.tight_layout()

            # Create buttons
            A_button = widgets.Button(description="A")
            B_button = widgets.Button(description="B")
            C_button = widgets.Button(description="C")
            D_button = widgets.Button(description="D")
    
            # List of buttons for easy access
            buttons = [A_button, B_button, C_button, D_button]
    
            # Add button functionality - changing windows
            A_button.on_click(handle_button_click("A"))
            B_button.on_click(handle_button_click("B"))
            C_button.on_click(handle_button_click("C"))
            D_button.on_click(handle_button_click("D"))

            # Pack buttons A, B, C, D into a horizontal box
            hbox_buttons = widgets.HBox([A_button, B_button, C_button, D_button],
                                layout=widgets.Layout(justify_content='center'))

            hbox_timer = widgets.HBox([self.timer.test_timer_widget],
                                layout=widgets.Layout(justify_content='center'))

            # Vertical box to position the buttons at the bottom and center horizontally
            vbox = widgets.VBox([hbox_buttons, hbox_timer], layout=widgets.Layout(
                                align_items='center',  # Center horizontally
                                justify_content='flex-end', # Align to bottom
                                width='100%'))  # Fill the width of the container

            # Display the vbox and the plot
            display(fig)
            display(vbox)

            #start the timer
            self.timer.puzzle_timer_manager("start")

#This class is responsible for puzzle creation logic and creating the puzzles using this logic.
class PuzzleCreator:
    def __init__(self):
        self.correctness = {}
 
    def draw_single_cube(self, cubes, ticks=False, grid=False, view='', flip='', rot=0, ax3d=None):
        #this is the logic for creating new cube arrangements
        # Create empty cube
        cubes_to_draw = np.zeros(cubes.shape)

        # Set elements to 1 where colour is not empty
        cubes_to_draw[cubes != ''] = 1

        # Set up axes for plotting
        ax = ax3d if ax3d is not None else plt.figure().add_subplot(projection='3d', proj_type='ortho', box_aspect=(4, 4, 4))

        nx, ny, nz = cubes.shape
        ax.axes.set_xlim3d(0, nx)
        ax.axes.set_ylim3d(0, ny)
        ax.axes.set_zlim3d(0, nz)

        # Plotting the cubes using a 3D voxels plot
        ax.voxels(cubes_to_draw, facecolors=cubes, edgecolors='k', shade=False)

        # Setting the 2D projection view
        if view == 'xy':
            ax.view_init(90, -90, 0 + rot)
        elif view == '-xy':
            ax.view_init(-90, 90, 0 - rot)
        elif view == 'xz':
            ax.view_init(0, -90, 0 + rot)
        elif view == '-xz':
            ax.view_init(0, 90, 0 - rot)
        elif view == 'yz':
            ax.view_init(0, 0, 0 + rot)
        elif view == '-yz':
            ax.view_init(0, 180, 0 - rot)
        else:
            ax.view_init(azim=ax.azim + rot)

        # Handling the flip argument for mirror image
        if 'x' in flip: ax.axes.set_xlim3d(nx, 0)
        if 'y' in flip: ax.axes.set_ylim3d(ny, 0)
        if 'z' in flip: ax.axes.set_zlim3d(nz, 0)

        # Styling figure ticks and grid lines
        if not ticks:
            for axis in [ax.xaxis, ax.yaxis, ax.zaxis]:
                axis.set_ticklabels([])
                axis.line.set_linestyle('')
                axis._axinfo['tick']['inward_factor'] = 0.0
                axis._axinfo['tick']['outward_factor'] = 0.0

        if not grid and not ticks:
            ax.set_axis_off()

        # If ax3d is provided, return the modified ax3d
        if ax3d is not None:
            return ax3d
        else:
            # If ax3d is not provided, show the plot
            plt.show()
            plt.close(ax.figure)

    def create_puzzle(self, puzzle_number, axes):
        #this is creating the actual puzzles we want to display
        fig = plt.figure(figsize=(20, 6))
        gs = gridspec.GridSpec(2, 4, figure=fig)

        if puzzle_number == 1:
            cubes = np.full((5, 5, 5), '')
            cubes[0:3,0,0] = 'r' 
            cubes[3,2:4,0] = 'g' 
            cubes[1:3,1:4,1:2] = 'b' 
            cubes[0:2,2,2] = 'y'

            cubes2 = np.full((5, 5, 5), '')
            cubes2[0:4, 0, 0] = 'r'  
            cubes2[1:3, 1:4, 1:2] = 'b'  
            cubes2[0:2, 2, 2] = 'y'


            axes[0].set_title("3D image", fontsize=36)
            axes[1].set_title("A", fontsize=20)
            axes[2].set_title("B", fontsize=20)
            axes[3].set_title("C", fontsize=20)
            axes[4].set_title("D", fontsize=20)

            self.draw_single_cube(cubes, ax3d=axes[0])
            self.draw_single_cube(cubes, view='xz', ax3d=axes[1])
            self.draw_single_cube(cubes, view='-xy', ax3d=axes[2])
            self.draw_single_cube(cubes, view='yx', rot=90, ax3d=axes[3])
            self.draw_single_cube(cubes2, flip="-yz", view='xz', rot=180, ax3d=axes[4])

            self.correctness  = {
                "A": 0,
                "B": 0,
                "C": 0,
                "D": 1
            }

        elif puzzle_number == 2:
            cubes = np.full((5, 5, 5), '')
            cubes[1, 1:5, 0] = 'r'
            cubes[0, 1, :] = 'b'
            cubes[0, :, 1] = 'g'

            cubes2 = np.full((5, 5, 5), '')
            cubes2[1, 1:5, 0] = 'r'
            cubes2[0, 1:4, :] = 'b'
            cubes2[0, :, 1] = 'g'

            axes[0].set_title("3D image", fontsize=36)
            axes[1].set_title("A", fontsize=20)
            axes[2].set_title("B", fontsize=20)
            axes[3].set_title("C", fontsize=20)
            axes[4].set_title("D", fontsize=20)

            self.draw_single_cube(cubes, rot=180, ax3d=axes[0])
            self.draw_single_cube(cubes, view='xy', ax3d=axes[1])
            self.draw_single_cube(cubes2, flip="-zy", view='xy', ax3d=axes[2])
            self.draw_single_cube(cubes, view='-yz', rot=-90, ax3d=axes[3])
            self.draw_single_cube(cubes, view='xz', ax3d=axes[4])

            self.correctness  = {
                "A": 0,
                "B": 1,
                "C": 0,
                "D": 0
            }

        elif puzzle_number == 3:
            cubes = np.full((5, 5, 5), '')
            cubes[0:3,0,0] = 'r' 
            cubes[3,1:3,0:2] = 'g' 
            cubes[1:3,1:4,0:1] = 'b'

            cubes2 = np.full((5, 5, 5), '')
            cubes2[0:3,0,0] = 'r' 
            cubes2[0,1:3,0:2] = 'g' 
            cubes2[1:3,1:4,0:1] = 'b'
            

            axes[0].set_title("3D image", fontsize=36)
            axes[1].set_title("A", fontsize=20)
            axes[2].set_title("B", fontsize=20)
            axes[3].set_title("C", fontsize=20)
            axes[4].set_title("D", fontsize=20)

            self.draw_single_cube(cubes, rot=-90, ax3d=axes[0])
            self.draw_single_cube(cubes, view='yz', ax3d=axes[1])
            self.draw_single_cube(cubes, view='xy', ax3d=axes[2])
            self.draw_single_cube(cubes2, flip="y", view='-xy', rot=-90, ax3d=axes[3])
            self.draw_single_cube(cubes, view='-xz',rot=180, ax3d=axes[4])

            self.correctness  = {
                "A": 0,
                "B": 0,
                "C": 1,
                "D": 0
            }

        elif puzzle_number == 4:
            cubes = np.full((5, 5, 5), '')
            cubes[:, 1:3, 1:3] = 'r'
            cubes[1:3, :, 1:3] = 'b'
            cubes[4, 0, :] = 'g'
            cubes[0, 3, :] = 'y'

            cubes2 = np.full((5, 5, 5), '')
            cubes2[:, 1:3, 1:3] = 'r'
            cubes2[1:3, :, 1:3] = 'b'
            cubes2[3:5, 0, :] = 'g'
            cubes2[0, 3, :] = 'y'

            axes[0].set_title("3D image", fontsize=36)
            axes[1].set_title("A", fontsize=20)
            axes[2].set_title("B", fontsize=20)
            axes[3].set_title("C", fontsize=20)
            axes[4].set_title("D", fontsize=20)

            self.draw_single_cube(cubes, rot=-90, ax3d=axes[0])
            self.draw_single_cube(cubes, view='-xy', ax3d=axes[1])
            self.draw_single_cube(cubes, view='-xz', ax3d=axes[2])
            self.draw_single_cube(cubes, view='-xy', rot=-90, ax3d=axes[3])
            self.draw_single_cube(cubes2, flip="x", view='-xz',rot=-90, ax3d=axes[4])

            self.correctness = {
                "A": 0,
                "B": 0,
                "C": 0,
                "D": 1
            }

        elif puzzle_number == 5:
            cubes = np.full((5, 5, 5), '')
            cubes[2, 2, :] = 'r'
            cubes[1, 1, 0:3] = 'b'
            cubes[1, 3, 0:2] = 'g'
            cubes[2:5, 0:2, 2] = 'y'
            cubes[0, 0, 0] = 'c'

            cubes2 = np.full((5, 5, 5), '')
            cubes2[2, 2, :] = 'r'
            cubes2[1, 1, 0:3] = 'b'
            cubes2[1, 3, 0:2] = 'g'
            cubes2[2:5, 0:3, 2] = 'y'
            cubes2[0, 0, 0] = 'c'
            
            axes[0].set_title("3D image", fontsize=36)
            axes[1].set_title("A", fontsize=20)
            axes[2].set_title("B", fontsize=20)
            axes[3].set_title("C", fontsize=20)
            axes[4].set_title("D", fontsize=20)

            self.draw_single_cube(cubes, rot=-90, ax3d=axes[0])
            self.draw_single_cube(cubes2, view='yz', rot=180, ax3d=axes[1])
            self.draw_single_cube(cubes, view='xy', ax3d=axes[2])
            self.draw_single_cube(cubes2, flip="z", view='yz', rot=180, ax3d=axes[3])
            self.draw_single_cube(cubes, view='-xz', ax3d=axes[4])
            
            self.correctness  = {
                "A": 1,
                "B": 0,
                "C": 0,
                "D": 0
            }

window_manager = WindowManager()
window_manager.whichpanel()

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…